In [1]:
import pandas as pd

from src.models import pooled_proba_model
from src.models import pooled_regression_model
from src.models import quote_regression_model

from src.features import data_for_modelling

# get data
orders_raw, offers_raw = data_for_modelling.get_raw_data()

# clean data
_, _, merged = data_for_modelling.clean_all(orders_raw, offers_raw)

# split offers into train test
merged_train, merged_test = data_for_modelling.split_train_test(merged)

# pickle data
merged_train.to_pickle("./data/pickels/merged_train.pkl") 
merged_test.to_pickle("./data/pickels/merged_test.pkl") 

# # read pickled data
# merged_train = pd.read_pickle("./data/pickels/merged_train.pkl") 
# merged_test = pd.read_pickle("./data/pickels/merged_test.pkl") 

In [15]:
merged_test.sort_values(by=["REFERENCE_NUMBER", "CREATED_ON_HQ"], ascending=True)

pooled_proba_pl, (df_X_test, df_y_test) = pooled_proba_model.generate_probability_pipeline(merged_train)
pooled_reg_pl, (df_X_test, df_y_test) = pooled_regression_model.generate_pooled_regression_pipeline(merged_train)
quote_reg_pl, (df_X_test, df_y_test) = quote_regression_model.generate_quote_regression_pipeline(merged_train)

probas = pooled_proba_pl.predict_proba(merged_test).T[0]
pooled_cost = pooled_reg_pl.predict(merged_test)
quote_cost = quote_reg_pl.predict(merged_test)

weighted_cost = probas * pooled_cost  + (1-probas) *  quote_cost

In [16]:
merged_test["ESTIMATED_COST_NEW"] = weighted_cost

In [19]:
merged.head(1)

,CARRIER_ID,REFERENCE_NUMBER,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,LOAD_DELIVERED_FROM_OFFER,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,...,ORDER_DAY,ORDER_MONTH,ORDER_HOUR,PICKUP_DAY,PICKUP_MONTH,PICKUP_HOUR,REMAINIG_TIME,BUSINESS_HOURS,BUSINESS_HOURS_ORDER_PICKUP,PRORATED_RATE_USD
0,0629d63476c157a6cfe234f7a34422eb0ad94998c2d082...,ce3548db155049cc1ccce2da041cec607942e4f779fc2d...,2021-11-03 08:54:13,9546.0,pool,False,2021-11-02 12:56:49,2021-11-03 16:00:00,945,601,...,1,11,12,2,11,16,25547.0,True,13.053056,795.5
